##INSAID Capstone project - group 1015

###Installing and importing mysql.connector/python

In [1]:
!pip install --user mysql-connector-python

In [0]:
import sys
import mysql.connector as sql
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling as pp
from google.colab import files

%matplotlib inline                                                              

import warnings                                                                 # Ignore warning related to pandas_profiling
warnings.filterwarnings('ignore') 

pd.set_option('display.max_columns', 100)                                       # Display all dataframe columns in outputs (it has 27 columns, which is wider than the notebook)
                                                                                # This sets it up to display with a horizontal scroll instead of hiding the middle columns

###Creating db connection

In [0]:
capdb = sql.connect(host='cpanel.insaid.co', database='Capstone1', user='student', password='student')

###Pulling data from SQL to Pandas df

In [4]:
print(pd.read_sql("show tables",con=capdb))

        Tables_in_Capstone1
0               events_data
1          gender_age_train
2  phone_brand_device_model


In [5]:
print(pd.read_sql("describe events_data",con=capdb))

       Field         Type Null Key Default Extra
0   event_id   bigint(20)   NO        None      
1  device_id  bigint(255)  YES        None      
2  timestamp     datetime   NO        None      
3  longitude   float(8,6)  YES        None      
4   latitude   float(8,6)  YES        None      
5       city  varchar(50)   NO        None      
6      state  varchar(50)   NO        None      


In [6]:
print(pd.read_sql("describe gender_age_train",con=capdb))

       Field           Type Null Key Default Extra
0  device_id    bigint(255)   NO        None      
1     gender  varchar(2555)   NO        None      
2        age       int(255)   NO        None      
3      group  varchar(2555)   NO        None      


In [7]:
print(pd.read_sql("describe phone_brand_device_model",con=capdb))

          Field           Type Null Key Default Extra
0     device_id    bigint(255)   NO        None      
1   phone_brand  varchar(2555)   NO        None      
2  device_model  varchar(2555)   NO        None      


In [8]:
query = """select *
          from events_data
          where state = 'delhi'"""
events_data_delhi = pd.read_sql(query, con=capdb)
events_data_delhi.shape

(751733, 7)

In [9]:
events_data_delhi.dtypes

event_id              int64
device_id           float64
timestamp    datetime64[ns]
longitude           float64
latitude            float64
city                 object
state                object
dtype: object

In [10]:
query = """select *
          from gender_age_train"""
gender_age_train = pd.read_sql(query, con=capdb)
gender_age_train.shape

(74645, 4)

In [11]:
query = """select *
          from phone_brand_device_model"""
phone_brand_device_model = pd.read_sql(query, con=capdb)
phone_brand_device_model.shape

(87726, 3)

In [0]:
pp.ProfileReport(events_data_delhi).to_file(outputfile='events_profile.html')
pp.ProfileReport(gender_age_train).to_file(outputfile='gender_age_train_profile.html')
pp.ProfileReport(phone_brand_device_model).to_file(outputfile='phone_brand_device_model.html')

files.download('events_profile.html')
files.download('gender_age_train_profile.html')
files.download('phone_brand_device_model.html')

In [0]:
evnt_join_gndr = pd.merge(left = events_data_delhi,
                  right = gender_age_train,
                  how ='left',
                  left_on = 'device_id',
                  right_on = 'device_id',
                  sort = False,
                  suffixes = ('_evnt','_gndr'),
                  indicator='ind_evnt_join_gndr',
                  validate = ('many_to_one')
                  )

In [14]:
pp.ProfileReport(evnt_join_gndr).to_file(outputfile='evnt_join_gndr.html')
files.download('evnt_join_gndr.html')

NameError: ignored

In [0]:
phone_brand_device_model.rename(columns = {'device_id':'device_id_mdl'}, inplace = True) 

capstone_data = pd.merge(left = df_merge_1,
                         right = phone_brand_device_model,
                         how ='left',
                         left_on = 'device_id_evnt',
                         right_on = 'device_id_mdl',
                         sort = False,
                         suffixes = ('_evntgndr','_mdl'),
                         indicator='ind_evntgndr_join_mdl',
                         validate = ('many_to_one')
                         )

### Closing db connection

In [0]:
capdb.close()